---
## **NOTA:** Toda la información producida durante el proyecto esta disponible en **https://drive.google.com/drive/folders/1v6ttK1kLlbDPv3gwIl_qY6QtO9HhZjMk?usp=sharing** el drive de Google: las bases de datos con los tweets descargados y procesados se encuentran como **archivos CSV** zipeados, asi como tambien los modelos de NLP (serializados con pickle).
---
#  **TA_POLITICA-01_prepara_scrapping.ipynb**
>## ENTRADA:
>## **"lista_politicos.csv"** : 
>## referencia compilada _a mano_ de **103 usuarios** de twitter que se van a escrapear:  Por cada _**@nombre_usuario**_ en la lista se crea el archivo **"tweetsdb_@nombre_usario.csv"** (Toda la data recuperada se encuentra archivada como **"tweets_politicos_db.zip"** )
>## **politicos_target.csv** : 
>## por cada usuario compilado , se _anotó_ el **partido** al que pertenecen ( o nul si no tiene afiliación ) y la **orientación** política personal o del partido. Se trato de elegir usuario públicos con un mínimo de 1000 seguidores (solo 8 de 103 usuario tienen menos de 1000 seguidores)
---
>## SALIDA:
>## **"tweetsdb_@nombre_usario.csv"**  : ( drive: **tweets_politicos_db.zip** ) todos los tweets por cada @nombre_usuario
>## **"politicosdb.csv"** : ( drive: **politicos_lista_anotada.zip** )informacíon detallada de cada @nombre_usuario , como _fecha_creacion_de_la_cuenta_ , _tweets_totales_ , _numero_de_seguidores_

In [6]:
from platform import python_version
print(python_version())

3.7.7


In [7]:
import pandas as pd
import GetOldTweets3 as got
import tweepy
import twitter
import json
import re
import sys
import time
import random
import pickle
from datetime import datetime,timedelta, date
sys.version

'3.7.7 (default, Apr 15 2020, 05:09:04) [MSC v.1916 64 bit (AMD64)]'

## Twitter API para recuperar la informacion de cada usuario del archivo **politicosdb.csv**
>### Esa información la usamos luego para recuperar los tweets con Tweepy y luego
>### con GetOldTweets3 hasta la fecha de creación de la cuenta

In [8]:
# Cargara las credenciales 
with open("twitter_credentials.json", "r") as file:
    creds = json.load(file)

In [9]:
api = twitter.Api(consumer_key=creds['CONSUMER_KEY'],
                  consumer_secret=creds['CONSUMER_SECRET'] ,
                  access_token_key=creds['ACCESS_TOKEN'],
                  access_token_secret=creds['ACCESS_SECRET'])

In [10]:
auth = tweepy.OAuthHandler(consumer_key=creds['CONSUMER_KEY'], consumer_secret=creds['CONSUMER_SECRET'])
auth.set_access_token(creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])
apiTweepy = tweepy.API(auth)

In [11]:
lista_politicos_csv = "lista_politicos.csv"
db_politicos_csv = "politicosdb.csv"

In [12]:
politicos_df = pd.read_csv(lista_politicos_csv)
politicos_df

,usuario_id
0,@MarthaChavezC
1,@Vero_Mendoza_F
2,@OmarChehadeMoya
3,@JuanSheput
4,@DanielUrresti1
...,...
101,@HumbertoTYL
102,@rogelio_tucto
103,@RichardArcePeru
104,@ManuelDammert


In [ ]:

politicosdb_list = []
i=1
itot=len(politicos_df)
for scr_name in politicos_df['usuario_id']:
    usuario=api.GetUser(user_id=None, screen_name=scr_name[1:], include_entities=False, return_json=False)
    fecha_creado = datetime.strptime(usuario.created_at, '%a %b %d %H:%M:%S +%G %Y').strftime("%Y-%m-%d")
    politicosdb_list.append([ usuario.id, scr_name , usuario.name, usuario.statuses_count, 0 , fecha_creado, fecha_creado, usuario.description, usuario.followers_count])
    print(f"Avance:{i}/{itot} Usuario:{scr_name}")
    i+=1
politicosdb_df = pd.DataFrame( politicosdb_list,
                   columns=['id','screen_name', 'nombre', 'total_tweets', 'tweets', 'fecha_creado' , 'fecha_min','descripcion', 'num_seguidores']  )

politicosdb_df.to_csv(db_politicos_csv, index=False)

Avance:1/106 Usuario:@MarthaChavezC
Avance:2/106 Usuario:@Vero_Mendoza_F
Avance:3/106 Usuario:@OmarChehadeMoya
Avance:4/106 Usuario:@JuanSheput
Avance:5/106 Usuario:@DanielUrresti1
Avance:6/106 Usuario:@anajarav
Avance:7/106 Usuario:@jtownsenddc
Avance:8/106 Usuario:@IndiraHuilca
Avance:9/106 Usuario:@UrsulaMoscoso
Avance:10/106 Usuario:@DianaMiloslavic
Avance:11/106 Usuario:@VicAZeballos
Avance:12/106 Usuario:@AlbertoBelaunde
Avance:13/106 Usuario:@leslye_lazo
Avance:14/106 Usuario:@LuisValdezF
Avance:15/106 Usuario:@ricardoburgaap
Avance:16/106 Usuario:@geovanniacate
Avance:17/106 Usuario:@EnriqueFCHA
Avance:18/106 Usuario:@arlettecontrers
Avance:19/106 Usuario:@MaricarmenAlvaP
Avance:20/106 Usuario:@FabioTr_
Avance:21/106 Usuario:@AleGarciaOviedo
Avance:22/106 Usuario:@SegundoMendezA
Avance:23/106 Usuario:@arturoayala_4
Avance:24/106 Usuario:@GeneralDelAire1
Avance:25/106 Usuario:@NormaValverdeR1
Avance:26/106 Usuario:@sigridbazan
Avance:27/106 Usuario:@dsalaverryv
Avance:28/106 Usu

In [ ]:
politicosdb_df

### Usamos TWEEPY para recoger los últimos 3200 tweets.
### Creamos un archivo .csv de tweets por cada político.
### En el siguiente notebook agregamos los tweets mas antiguos a cada archivo .csv con GetOldTweets3

In [12]:
#usamos TWEEPY para recoger los ultimos 3200 tweets
#porque es más rápido que GetOldTweets3 (solo para los ultimos 3200)

hoy_str = date.today().strftime("%Y-%m-%d")

for idx,scr_name in enumerate(politicosdb_df['screen_name']):   

    nombre = politicosdb_df['nombre'].iloc[idx]
    creado = politicosdb_df['fecha_creado'].iloc[idx]
    total_tweets = politicosdb_df['total_tweets'].iloc[idx]

    cursor = tweepy.Cursor(apiTweepy.user_timeline, screen_name=scr_name,tweet_mode="extended",count=200)
    tweets_list = []
    i = 0
    print(f"Usuario:{scr_name} ({nombre}) / Creado: {creado} / Total_tweets:{total_tweets}")
    for tw in cursor.items():
        
        if tw.retweeted:
            _text = tw.retweeted_status.full_text 
        else:
            _text = tw.full_text

        text_tweet = [tw.id,
                        tw.user.screen_name,
                        _text,
                        f"{tw.created_at.year}-{tw.created_at.month}-{tw.created_at.day}" ,
                        tw.retweet_count,
                        ' '.join([f"@{txt['screen_name']}"  for txt in tw.entities['user_mentions']] ),
                        ' '.join([f"#{txt['text']}" for txt in tw.entities['hashtags']])
                       ]
        tweets_list.append(text_tweet)
        i += 1
        print(f"Tweets recuperados->{i:04d}", end ="\r")
        #print(text_tweet)
   
    end_date_str=tw.created_at.strftime("%Y-%m-%d")
    politicosdb_df['fecha_min'].iloc[idx]=end_date_str
    politicosdb_df.to_csv(db_politicos_csv)
    print(f"Tweets recuperados->{i:04d} del {hoy_str} hasta {end_date_str}")

    #Creamos un dataframe para guardarlo en el disco
    usuario_df = pd.DataFrame(tweets_list, 
                          columns = ['id','User', 'Text','Date', 'Retweets', 'Mentions', 'HashTags'])

    usuario_df.to_csv("tweetsdb_"+scr_name+".csv", index=False)
    print(len(usuario_df))


Usuario:@vozdelatierra (Marco Arana 🇵🇪) / Creado: 2011-08-24 / Total_tweets:72081


D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Tweets recuperados->3221 del 2020-06-26 hasta 2020-02-04
3221
Usuario:@GGG_pe (Gustavo Guerra García) / Creado: 2011-01-13 / Total_tweets:2555
Tweets recuperados->2539 del 2020-06-26 hasta 2011-01-13
2539
Usuario:@JorgeMunozAP (Jorge Muñoz) / Creado: 2010-07-06 / Total_tweets:5668
Tweets recuperados->3114 del 2020-06-26 hasta 2018-07-29
3114
Usuario:@MirtyVas (Mirtha Vásquez) / Creado: 2011-06-15 / Total_tweets:773
Tweets recuperados->0772 del 2020-06-26 hasta 2011-12-16
772
Usuario:@AntauroHumala (Antauro Humala) / Creado: 2011-11-06 / Total_tweets:331
Tweets recuperados->0331 del 2020-06-26 hasta 2011-12-09
331
Usuario:@HCevallosFlores (Hernando Cevallos) / Creado: 2016-08-09 / Total_tweets:1701
Tweets recuperados->1684 del 2020-06-26 hasta 2016-08-19
1684
Usuario:@FrenteAmplioPe (Frente Amplio Perú) / Creado: 2017-12-22 / Total_tweets:7088
Tweets recuperados->3216 del 2020-06-26 hasta 2018-06-04
3216
Usuario:@AccionPopular (Acción Popular) / Creado: 2009-08-20 / Total_tweets:3151
Tw

---
## El trabajo continua en el notebook   **TA_POLITICA-02_twitterscrapping.ipynb** (GetOldTweets3)


In [1]:
!dir tweetsdb*

 El volumen de la unidad C es OS
 El n£mero de serie del volumen es: 1823-0BBE

 Directorio de C:\Users\Facundo\Downloads\Diplomado IAPUCP\2ASRS\TAREA_ACADEMICA

26/06/2020  14:31    <DIR>          tweetsdb
26/06/2020  08:36           634,401 tweetsdb_@aaronsalomong.csv
26/06/2020  16:51           650,732 tweetsdb_@AlanGarciaPeru.csv
26/06/2020  16:47           752,233 tweetsdb_@AlbertoBelaunde.csv
26/06/2020  16:53            93,166 tweetsdb_@albertofujimori.csv
26/06/2020  08:43           731,155 tweetsdb_@AlcortaLourdes.csv
26/06/2020  16:47           446,739 tweetsdb_@AleGarciaOviedo.csv
26/06/2020  16:53           762,918 tweetsdb_@amariateguinof.csv
26/06/2020  16:47           918,136 tweetsdb_@anajarav.csv
26/06/2020  08:43           157,185 tweetsdb_@APRAXXI.csv
26/06/2020  16:47           722,679 tweetsdb_@arlettecontrers.csv
26/06/2020  08:09           709,267 tweetsdb_@arturoayala_4.csv
26/06/2020  16:53           695,400 tweetsdb_@A_Aguinaga.csv
26/06/2020  08:38           